<a href="https://colab.research.google.com/github/czhurdlespeed/TFDevCert/blob/main/TFCert_Practice/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

## Check for GPU

In [192]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-756ae205-fd00-a725-b1b7-9297870d5bb7)


## Import Helper Functions

In [193]:
!wget https://github.com/mrdbourke/tensorflow-deep-learning/raw/main/extras/helper_functions.py
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-05-09 13:20:12--  https://github.com/mrdbourke/tensorflow-deep-learning/raw/main/extras/helper_functions.py
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py [following]
--2024-05-09 13:20:12--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.4’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-05-09 13:20:12 (89.3 MB/s) - ‘helpe

## Get a text dataset

- Using Kaggle's Introduction to NLP dataset

In [194]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
# Uzip data
unzip_data("nlp_getting_started.zip")

--2024-05-09 13:20:12--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.118.207, 74.125.200.207, 74.125.130.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.118.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.4’

nlp_getting_started 100%[===================>] 593.11K   734KB/s    in 0.8s    

2024-05-09 13:20:14 (734 KB/s) - ‘nlp_getting_started.zip.4’ saved [607343/607343]



## Visualizing a text dataset

In [195]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [196]:
train_df["text"][100]

'.@NorwayMFA #Bahrain police had previously died in a road accident they were not killed by explosion https://t.co/gFJfgTodad'

In [197]:
# Shuffle training datafram
train_df_shuffled = train_df.sample(frac=1)
train_df_shuffled.head()

,id,keyword,location,text,target
7258,10392,whirlwind,Frostburg,HELP I'M IN A WHIRLWIND OF NOSTALGIA,0
5388,7688,panic,"Macon, GA",Romantic dramatic but never panic original sen...,0
1667,2409,collide,"Melton, GA",Somehow find you and I collide http://t.co/Ee8...,0
380,546,arson,NaN,Two Jewish Terrorists Charged In Historic-Chur...,0
2461,3529,derailment,NaN,#???? #?? #??? #??? Trains derailment: 'It's t...,1


In [198]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [199]:
# How many examples of each class are there?
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [200]:
# How many locations?
train_df.location.value_counts()

location
USA                    104
New York                71
United States           50
London                  45
Canada                  29
                      ... 
Some pum pum             1
Paulton, England         1
SEA Server               1
todaysbigstock.com       1
???????? ?????????.      1
Name: count, Length: 3341, dtype: int64

In [201]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [202]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", r"(real disaster)" if target > 0 else r"(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
Trauma injuries involving kids and sport usually cycling related: Director Trauma NS  http://t.co/8DdijZyNkf #NS http://t.co/52Uus4TFN3

---

Target: 1 (real disaster)
Text:
Nuclear #Solar Power #Japanese #Fukushima Reactor Energy Japan Temperature No. Fuel Pool More  http://t.co/YS3nMwWyVc http://t.co/AlpotNB7q3

---

Target: 0 (not real disaster)
Text:
Let's destroy Twitter with @fouseyTUBE @zaynmalik come on BrBrS lets do it in morning with fousey

---

Target: 1 (real disaster)
Text:
How One Couple Is Using Drones to Save Refugees at the World's Deadliest Border http://t.co/9qpG0Z3Rh9

---

Target: 0 (not real disaster)
Text:
how will I survive without dorrian

---



In [203]:
print("Hello\'\\ \n")

Hello'\ 



In [204]:
print(r"Hello\'\\ \n")

Hello\'\\ \n


### Split data into training and validation sets

In [205]:
from sklearn.model_selection import train_test_split
train_sentences, validation_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                                   train_df_shuffled["target"].to_numpy(),
                                                                                   test_size=0.1)


In [206]:
# Check the lengths
len(train_sentences), len(train_labels), len(validation_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [207]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['Police officer wounded suspect dead after exchanging shots: RICHMOND Va. (AP) \x89ÛÓ A Richmond police officer wa... http://t.co/Y0qQS2L7bS',
        'USGS EQ: M 1.9 - 15km E of Anchorage Alaska: Time2015-08-06 00:11:16 UTC2015-08-05 16:11:16 -08:0... http://t.co/OjQ0KFg5ub #EarthQuake',
        'drake been kept it the most hip hop during this beef and he fucking body bagging meek back to back ??',
        "http://t.co/kG5pLkeDhr WRAPUP 2-U.S. cable TV companies' shares crushed after Disney disappoints http://t.co/QeIhvn3DNQ",
        'HamptonRoadsFor.me US Navy Sidelines 3 Newest Subs - http://t.co/9QNQ45Zduw http://t.co/dhyLJllRHL',
        'Well me and dad survived my driving ????????',
        '#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires',
        "Cramer: Iger's 3 words that wrecked Disney's stock http://t.co/3G79prAyYc #cnbc #topnews",
        'Suffield Alberta Accident https://t.co/bPTmlF4P10',
        '

## Converting text into numbers

In [208]:
# Embedding - every word gets turned into a vector; you can specify the dimensions of the vector
  # gets updated as the network learns; therfore it can learn relationships between tokens
  # Pretrained embedding libraries e.g. word2vec, GloVe, etc.


In [209]:
# Text vectorization (tokenization)
train_sentences[:5]

array(['Police officer wounded suspect dead after exchanging shots: RICHMOND Va. (AP) \x89ÛÓ A Richmond police officer wa... http://t.co/Y0qQS2L7bS',
       'USGS EQ: M 1.9 - 15km E of Anchorage Alaska: Time2015-08-06 00:11:16 UTC2015-08-05 16:11:16 -08:0... http://t.co/OjQ0KFg5ub #EarthQuake',
       'drake been kept it the most hip hop during this beef and he fucking body bagging meek back to back ??',
       "http://t.co/kG5pLkeDhr WRAPUP 2-U.S. cable TV companies' shares crushed after Disney disappoints http://t.co/QeIhvn3DNQ",
       'HamptonRoadsFor.me US Navy Sidelines 3 Newest Subs - http://t.co/9QNQ45Zduw http://t.co/dhyLJllRHL'],
      dtype=object)

In [210]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# use the default textvectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in our vocabulary; no cap on size of vocab
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, # create groups of n-words?
                                    output_mode='int', # how to map tokens to numbers
                                    output_sequence_length=None,
                                    pad_to_max_tokens=False)

In [211]:
# find the average number of tokens in the training tweets
round((sum([len(i.split()) for i in train_sentences]))/len(train_sentences))

15

In [212]:
# Setup text vectorization variables
max_vocab_length = 10000 # max num of words to have in vocab
max_length = 15 # max length our sequences will be

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [213]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [214]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[272,   3, 217,   4,  13, 674,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [215]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence} \
      \n\nVectorized versions:")
text_vectorizer([random_sentence])

Original text:
@thebriankrause leos ass just got metaphorically blown up again #PiperWearsThePants #charmed       

Vectorized versions:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[7673,    1,  282,   31,   98,    1,  447,   27,  308, 9791,    1,
           0,    0,    0,    0]])>

In [216]:
# Get the unique words in the vocab
words_in_vocab = text_vectorizer.get_vocabulary() # Get all the unique words in our training data
# Most common words
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['palmoil', 'palmer', 'palm', 'palinfoen', 'palestinian\x89Û']


### Creating an Embedding using an Embedding Layer

In [217]:
# taken tokenized text into embedding vectors
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128, # best to choose number divisible by 8
                             input_length=max_length # how long is each input
                             )
embedding

In [218]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
\n\n Embedded version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 #gunsense answer to #GFZ's: break law on the minuscule chance a mass murderer shows up. https://t.co/qEoPMCJbCz

 Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00443272,  0.02111781, -0.01387938, ..., -0.04988755,
         -0.01515515,  0.02236426],
        [ 0.00678701, -0.0131033 , -0.02599986, ..., -0.00091382,
          0.03215111, -0.03315163],
        [ 0.04364384,  0.04539366, -0.00457828, ...,  0.02578819,
          0.01497266, -0.02823187],
        ...,
        [-0.03106868, -0.00853967,  0.04041448, ..., -0.04214721,
          0.00964229, -0.04934539],
        [-0.02664727, -0.03528628,  0.005102  , ..., -0.00845378,
          0.00966469,  0.03582777],
        [-0.01802797, -0.0232386 , -0.00928166, ...,  0.00289299,
         -0.04738591, -0.00303062]]], dtype=float32)>

In [219]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.00443272,  0.02111781, -0.01387938,  0.04608704, -0.02822477,
         0.0243644 , -0.0276803 , -0.00337965,  0.01049309,  0.04351186,
        -0.00029291, -0.0350575 ,  0.00087998,  0.03165218, -0.0334216 ,
         0.0009285 ,  0.01226233, -0.02607198, -0.04187089, -0.0002422 ,
         0.01096771, -0.03312057,  0.01002539, -0.00879133, -0.01457537,
         0.04797632,  0.00966503, -0.00012533, -0.04255216, -0.0278367 ,
        -0.0312018 , -0.00370036, -0.04241985, -0.00599625, -0.0280579 ,
         0.04652428,  0.03206017, -0.01588542, -0.01974161,  0.04693463,
        -0.04283508, -0.00485136, -0.02641841,  0.04658547, -0.01334356,
         0.0266858 , -0.01892482,  0.02475021,  0.004844  , -0.02822222,
         0.04765535,  0.04108945, -0.04910952,  0.02943556,  0.00812332,
         0.04991331, -0.00473022,  0.02039618,  0.03090305, -0.02925174,
        -0.03323587,  0.00065473, -0.01156052,  0.02172066, -0.04945525,
  

## Modeling a text dataset
* Model 0: Naive Bayes (baseline) [sklearn ML map](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional NN (CNN)
* Model 6: TF Hub Pretrained Feature Extractor (transfer learning)
* Model 7: Same as model 6 with 10% of training data

### Model 0: Getting a baseline

In [220]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer # turn text into numbers/vectors
from sklearn.pipeline import Pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB()) # clf == "classifier"
])
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [221]:
# Evaluate our baseline model
baseline_score = model_0.score(validation_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 78.87%


In [222]:
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [223]:
# Make predictions
baseline_preds = model_0.predict(validation_sentences)
baseline_preds[:20]

array([0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1])

### Creating an evaluation function for our model experiments

Create on to compare our model's predictions with the truth labels
* Accuracy
* Precision
* Recall
* F1-score

In [224]:
# Function to evaluate: accuracy, precision, recall, F1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall, and f1-score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [225]:
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred = baseline_preds)
baseline_results

{'accuracy': 78.87139107611549,
 'precision': 0.8052727924775167,
 'recall': 0.7887139107611548,
 'f1': 0.7798859001389012}

# Model 1: Feed-forward neural network (dense model)

In [226]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save Tensorboard logs
SAVE_DIR = "model_logs"

# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1D strings
x = text_vectorizer(inputs) # turn input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x) # Create output layer, want binary outputs
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [227]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_9 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_3  (None, 128)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [228]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [229]:
# Fit model
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(validation_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20240509-132015
Epoch 1/5
215/215 [==============================] - 8s 36ms/step - loss: 0.6129 - accuracy: 0.6894 - val_loss: 0.5350 - val_accuracy: 0.7598
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4422 - accuracy: 0.8193 - val_loss: 0.4658 - val_accuracy: 0.7979
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3463 - accuracy: 0.8610 - val_loss: 0.4556 - val_accuracy: 0.8071
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2837 - accuracy: 0.8902 - val_loss: 0.4613 - val_accuracy: 0.8110
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2358 - accuracy: 0.9133 - val_loss: 0.4827 - val_accuracy: 0.7913


In [230]:
# Check the results
model_1.evaluate(validation_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4827 - accuracy: 0.7913


[0.48266905546188354, 0.7913385629653931]

In [231]:
# make some predictions and evaluate those
model_1_pred_probs = model_1.predict(validation_sentences)
model_1_pred_probs

24/24 [==============================] - 0s 2ms/step


array([[2.81842686e-02],
       [1.51438668e-01],
       [9.93940473e-01],
       [7.95873925e-02],
       [9.22450602e-01],
       [8.28504920e-01],
       [4.17281568e-01],
       [8.94918501e-01],
       [4.34865206e-02],
       [3.78188640e-02],
       [4.56923768e-02],
       [4.96448308e-01],
       [1.34916110e-02],
       [2.49537140e-01],
       [2.79540867e-01],
       [9.98828590e-01],
       [6.09150752e-02],
       [7.77264535e-02],
       [9.94072020e-01],
       [9.99042928e-01],
       [3.43884438e-01],
       [1.73049912e-01],
       [4.38854322e-02],
       [9.55291092e-01],
       [9.08613130e-02],
       [1.43668160e-01],
       [5.75585552e-02],
       [9.12534237e-01],
       [1.93220064e-01],
       [1.79354966e-01],
       [2.76168227e-01],
       [8.64032209e-02],
       [2.77697265e-01],
       [2.60457039e-01],
       [8.39035958e-02],
       [1.39375515e-02],
       [5.22520356e-02],
       [9.99909282e-01],
       [8.45079899e-01],
       [3.80610466e-01],


In [232]:
# Convert model prediction probabilites to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1.,
       1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1.,
       1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
       1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1.,
       1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0.,
       0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0

In [233]:
# Calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 79.13385826771653,
 'precision': 0.7912066204319435,
 'recall': 0.7913385826771654,
 'f1': 0.7893381707827568}

In [234]:
baseline_results

{'accuracy': 78.87139107611549,
 'precision': 0.8052727924775167,
 'recall': 0.7887139107611548,
 'f1': 0.7798859001389012}

In [235]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([ True, False,  True,  True])

## Visualizing learned embeddings

In [236]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [237]:
# Model 1 Summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_9 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_3  (None, 128)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [239]:
# Get the weight matrix of embedding layer
# (numerical representations of each token in our training data; learned for 5 epochs)
embed_weights = model_1.get_layer("embedding_3").get_weights()[0]
embed_weights

array([[-0.04403623, -0.00592298, -0.01087402, ...,  0.04359039,
        -0.01070671, -0.01374578],
       [-0.00128164, -0.00358714,  0.01813553, ..., -0.04749976,
        -0.0258109 , -0.00331443],
       [ 0.01426032, -0.05304237,  0.02409918, ...,  0.04333581,
         0.02374695, -0.02516862],
       ...,
       [-0.10387441, -0.03454635,  0.10201775, ...,  0.06870296,
        -0.02712727, -0.07230251],
       [-0.0118618 , -0.00940547, -0.06200809, ..., -0.08189034,
         0.02716805,  0.06245387],
       [-0.01136811, -0.01367552, -0.00688862, ..., -0.04250362,
        -0.03590056,  0.03451348]], dtype=float32)

In [240]:
print(embed_weights.shape) # same size as vocab size and embedding_dim; each word embedded into 128 dim vector

(10000, 128)


# [Tensorflow Embedding Visualization Tool](https://projector.tensorflow.org/)

In [241]:
# Create embedding files
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [242]:
# Download files from Colab to upload to Tensorflow projector (see link above)
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks (RNN's)
* Sequence data
* Use representation of previous input to aid the representation of a later input

### Model 2: LSTM
LSTM = long short term memory (one of th most popular Recurrent Cells)

RNN Structure:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (label probability)
```

In [243]:
# Create an LSTM Model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs) # map text to numbers/tokens
x = embedding(x) # create embedding vectors from tokens
#print(x.shape)
#x = layers.LSTM(units=64, return_sequences=True)(x) # returns a squence; maintains dims of a sequence
# when you're stacking RNN cells together, you need to set return_sequences=True
#   i.e. input to LSTM expected to be a sequence; the output doesn't necessarily have to be a sequence
#print(x.shape)
x = layers.LSTM(64)(x)
#print(x.shape)
#x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs, name="model_2_LSTM")

In [244]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_9 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [245]:
# Compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [246]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(validation_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20240509-132110
Epoch 1/5
215/215 [==============================] - 19s 54ms/step - loss: 0.2194 - accuracy: 0.9251 - val_loss: 0.5536 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.1491 - accuracy: 0.9457 - val_loss: 0.6532 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1277 - accuracy: 0.9543 - val_loss: 0.7286 - val_accuracy: 0.7612
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1022 - accuracy: 0.9625 - val_loss: 0.8977 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0875 - accuracy: 0.9667 - val_loss: 0.8772 - val_accuracy: 0.7690


In [247]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(validation_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[9.0721331e-04],
       [2.9540632e-03],
       [9.9941576e-01],
       [3.8344229e-03],
       [9.9908364e-01],
       [9.9694604e-01],
       [4.6275026e-01],
       [9.9655980e-01],
       [1.2545059e-03],
       [3.9470542e-04]], dtype=float32)

In [248]:
# convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 1., 1., 0., 1., 0., 0.], dtype=float32)>

In [249]:
# Calculate model 2 results
model_2_results = calculate_results(val_labels, model_2_preds)
model_2_results

{'accuracy': 76.9028871391076,
 'precision': 0.7679944435327405,
 'recall': 0.7690288713910761,
 'f1': 0.7678571029503982}

In [250]:
# But baseline is still better
baseline_results

{'accuracy': 78.87139107611549,
 'precision': 0.8052727924775167,
 'recall': 0.7887139107611548,
 'f1': 0.7798859001389012}

### Model 3: RNN with GRU Cell

In [251]:
# Build an RNN using GRU Cell
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
#print(x.shape)
x = layers.GRU(64)(x) # if stacking recurrent layers, return_sequences = True
# print(x.shape)
# x = layers.LSTM(42, return_sequences=True)(x)
# print(x.shape)
# x = layers.GRU(99)(x)
# print(x.shape)
#x = layers.Dense(64, activation='relu')(x)
#x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="Model_3_GRU")

In [252]:
model_3.summary()

Model: "Model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_9 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru_2 (GRU)                 (None, 64)                37248     
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1317313 (5.03 MB)
Trainable params: 1317313 (5.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________

In [253]:
# Compile
model_3.compile(loss="binary_crossentropy", optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [254]:
model_3_history= model_3.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(validation_sentences, val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR, "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20240509-132139
Epoch 1/5
215/215 [==============================] - 14s 53ms/step - loss: 0.1508 - accuracy: 0.9423 - val_loss: 0.7135 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0821 - accuracy: 0.9692 - val_loss: 0.8849 - val_accuracy: 0.7533
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0713 - accuracy: 0.9731 - val_loss: 0.9032 - val_accuracy: 0.7493
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0617 - accuracy: 0.9749 - val_loss: 1.0690 - val_accuracy: 0.7559
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0541 - accuracy: 0.9783 - val_loss: 1.1303 - val_accuracy: 0.7585


In [255]:
# Mae some predictions with our GRU model
model_3_pred_probs = model_3.predict(validation_sentences)
model_3_pred_probs

24/24 [==============================] - 1s 5ms/step


array([[5.77267492e-04],
       [5.96305239e-04],
       [9.99789178e-01],
       [1.93540379e-03],
       [9.99771178e-01],
       [9.98894632e-01],
       [3.89673263e-01],
       [9.99612391e-01],
       [4.49765800e-03],
       [2.40031295e-04],
       [2.96234590e-04],
       [9.53831434e-01],
       [1.82594551e-04],
       [1.77759692e-01],
       [1.26685509e-02],
       [9.99898076e-01],
       [2.00660108e-03],
       [2.33733444e-04],
       [9.99865413e-01],
       [9.99891877e-01],
       [1.28345013e-01],
       [1.57422770e-03],
       [2.51817517e-03],
       [9.99789417e-01],
       [4.00361186e-03],
       [1.57885402e-02],
       [3.38115904e-04],
       [9.99807179e-01],
       [4.90028760e-04],
       [2.31152698e-01],
       [9.89288688e-01],
       [9.49019042e-04],
       [9.52785969e-01],
       [1.53006345e-01],
       [2.78321877e-02],
       [6.43130916e-04],
       [2.92287441e-03],
       [9.99948382e-01],
       [9.99435484e-01],
       [1.32047357e-02],


In [256]:
# Convert pred probs ot labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1.,
       1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1.,
       1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
       1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1.,
       1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0.,
       0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0

In [257]:
# Calcualte model 3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 75.8530183727034,
 'precision': 0.7573698183160616,
 'recall': 0.7585301837270341,
 'f1': 0.7573051530845072}

## Model 4: Bidirecitonal RNN

Normal RNN's go from left to right

Bidirecitonal RNN's to right to left as well as left to right (picks up more context)

In [258]:
# Build a bidirectional RNN in TF
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs,outputs, name="model_4_bidirectional")


In [259]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_9 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional_2 (Bidirecti  (None, 15, 128)           98816     
 onal)                                                           
                                                                 
 bidirectional_3 (Bidirecti  (None, 128)               74496     
 onal)                                                           
                                             

In [260]:
model_4.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [261]:
model_4.fit(train_sentences,
            train_labels,
            epochs=5,
            validation_data=(validation_sentences,val_labels),
            callbacks=[create_tensorboard_callback(SAVE_DIR, "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20240509-132207
Epoch 1/5
215/215 [==============================] - 22s 51ms/step - loss: 0.0985 - accuracy: 0.9692 - val_loss: 1.0990 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0568 - accuracy: 0.9761 - val_loss: 1.2751 - val_accuracy: 0.7388
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0522 - accuracy: 0.9772 - val_loss: 1.1583 - val_accuracy: 0.7559
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0425 - accuracy: 0.9797 - val_loss: 1.6624 - val_accuracy: 0.7467
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0407 - accuracy: 0.9816 - val_loss: 1.3931 - val_accuracy: 0.7441


In [262]:
# Make predictions
model_4_pred_probs = model_4.predict(validation_sentences)


24/24 [==============================] - 1s 4ms/step


In [263]:
model_4_pred_probs[:10]

array([[1.3651466e-04],
       [4.8588987e-05],
       [9.9972671e-01],
       [3.7390983e-04],
       [9.9991536e-01],
       [9.9948198e-01],
       [5.4441667e-01],
       [9.9931121e-01],
       [5.1667187e-03],
       [2.9405694e-05]], dtype=float32)

In [264]:
# Convert to labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 1., 1., 1., 1., 0., 0.], dtype=float32)>

In [265]:
# Calculate results
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred = model_4_preds)
model_4_results

{'accuracy': 74.40944881889764,
 'precision': 0.7433536641678077,
 'recall': 0.7440944881889764,
 'f1': 0.7436191973741199}

##Convolutional NN for Text

Images are typically 2D; Text data is 1D


Inputs -> Tokenization -> Embedding -> Layers (Conv1D + pooling) -> Outputs

In [277]:
# Test out our embedding layer, Conv1D layer and max pooling
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn sequence into embedding
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5, # ngram of 5 (looks at 5 words at a time)
                        activation="selu",
                        padding="valid")
conv_1d_output= conv_1d(embedding_test) # pass test embedding layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # get feature with highest value

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [278]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.05843185,  0.01257363,  0.0255771 , ...,  0.05457251,
          0.00660438,  0.00676524],
        [-0.00160623, -0.00578824,  0.00590827, ...,  0.01145494,
         -0.00784337, -0.05313375],
        [-0.00765452,  0.00474321,  0.02777818, ..., -0.03914494,
         -0.03059993,  0.00791953],
        ...,
        [-0.01855001,  0.0042194 ,  0.00130567, ...,  0.03172339,
          0.00220184, -0.00983087],
        [-0.01855001,  0.0042194 ,  0.00130567, ...,  0.03172339,
          0.00220184, -0.00983087],
        [-0.01855001,  0.0042194 ,  0.00130567, ...,  0.03172339,
          0.00220184, -0.00983087]]], dtype=float32)>

In [279]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[ 0.06337401,  0.09514693,  0.06742668,  0.11024044,
          0.11831163, -0.05510854, -0.01610063, -0.17912196,
         -0.37229884, -0.14535685,  0.20468119, -0.0938577 ,
         -0.07284734, -0.00442498,  0.02525175,  0.00812321,
         -0.08725075,  0.00955595,  0.0178788 , -0.1113418 ,
         -0.03125086,  0.0600495 , -0.03785806, -0.07970775,
          0.0146082 ,  0.01274408,  0.08187455, -0.05860578,
         -0.01140439, -0.06076407, -0.21878418, -0.07229719],
        [ 0.07887485, -0.09647741, -0.06100278,  0.02115549,
         -0.14434229,  0.04320299,  0.04958136, -0.01305516,
         -0.11708956,  0.05402226,  0.044838  ,  0.02143831,
          0.10011204, -0.08960298, -0.11747881, -0.01845814,
         -0.07110558, -0.00038973, -0.02920645, -0.07144028,
         -0.03998177,  0.07057743, -0.05777311, -0.02156433,
          0.03270371, -0.13075936,  0.06968483, -0.32980305,
         -0.0785756 , -0.241632

In [281]:
# Create 1D convolutional layer to model sequences
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64,
                  kernel_size=5,
                  activation="relu",
                  padding="valid",
                  strides=1)(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="Model_5_Conv1D")

model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_5.summary()

Model: "Model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_9 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 conv1d_7 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_7 (Gl  (None, 64)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_10 (Dense)            (None, 1)              

In [282]:
model_5.fit(train_sentences,
            train_labels,
            epochs=5,
            validation_data = (validation_sentences, val_labels),
            callbacks=[create_tensorboard_callback(SAVE_DIR, "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20240509-134047
Epoch 1/5
215/215 [==============================] - 17s 66ms/step - loss: 0.1202 - accuracy: 0.9612 - val_loss: 1.0132 - val_accuracy: 0.7336
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0738 - accuracy: 0.9707 - val_loss: 1.1671 - val_accuracy: 0.7323
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0600 - accuracy: 0.9761 - val_loss: 1.3248 - val_accuracy: 0.7165
Epoch 4/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0542 - accuracy: 0.9777 - val_loss: 1.3384 - val_accuracy: 0.7283
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0488 - accuracy: 0.9788 - val_loss: 1.3773 - val_accuracy: 0.7283


In [283]:
model_5_pred_probs = model_5.predict(validation_sentences)
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

24/24 [==============================] - 0s 4ms/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 1., 1., 1., 1., 0., 0.], dtype=float32)>

In [284]:
model_5_results = calculate_results(y_true = val_labels,
                                    y_pred = model_5_preds)

In [285]:
model_5_results

{'accuracy': 72.83464566929135,
 'precision': 0.7284511102123249,
 'recall': 0.7283464566929134,
 'f1': 0.7283975234259907}

In [286]:
## Model 6: TF Hub Pretrained Sentence Encoder
import tensorflow_hub as hub

In [289]:
print(sample_sentence)

There's a flood in my street!


In [290]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample_sentence, "When you use the universal sentence encoder on a sentence, it turns the sentence into an embedding vector"])
print(embed_samples[0])

tf.Tensor(
[-0.01157028  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680984  0.05589836 -0.0106873  -0.00597291  0.00639323 -0.01819518
  0.00030813  0.09105888  0.05874644 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865347 -0.04209306  0.02678981  0.03011006  0.00321069
 -0.00337973 -0.04787357  0.0226672  -0.00985925 -0.04063613 -0.01292092
 -0.04666384  0.05630299 -0.03949255  0.00517686  0.02495829 -0.0701444
  0.02871508  0.04947684 -0.00633979 -0.08960192  0.02807118 -0.00808364
 -0.01360602  0.0599865  -0.10361787 -0.05195374  0.00232954 -0.02332531
 -0.03758105  0.03327728 -0.00430603 -0.05894244 -0.06101276 -0.02220005
 -0.01575761 -0.00474414 -0.03515062 -0.04440377 -0.04174353  0.04943503
 -0.02740932 -0.04133101  0.01634346 -0.03713681  0.07395207 -0.00659238
 -0.0694331  -0.00361752  0.08656701  0.07717378 -0.06633057  0.02086627
  0.06676423  0.00982106  0.01410716  0.07672232  0.05026635  0.02969931
 -0.01391874  0.06082514  0.08967174 -0.01

In [291]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", input_shape=[], dtype=tf.string, trainable=False, name="USE")


In [303]:
# Create model using Sequential API

model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])


In [304]:
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [305]:
model_6.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_16 (Dense)            (None, 64)                32832     
                                                                 
 dense_17 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [306]:
# Train classifier on top of USE pretrained embeddings
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data = [validation_sentences, val_labels],
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "tf_hub_sentence_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20240509-141232
Epoch 1/10
215/215 [==============================] - 10s 20ms/step - loss: 0.5066 - accuracy: 0.7831 - val_loss: 0.4562 - val_accuracy: 0.7979
Epoch 2/10
215/215 [==============================] - 3s 12ms/step - loss: 0.4128 - accuracy: 0.8174 - val_loss: 0.4488 - val_accuracy: 0.8084
Epoch 3/10
215/215 [==============================] - 3s 13ms/step - loss: 0.3988 - accuracy: 0.8222 - val_loss: 0.4448 - val_accuracy: 0.8097
Epoch 4/10
215/215 [==============================] - 4s 16ms/step - loss: 0.3898 - accuracy: 0.8272 - val_loss: 0.4449 - val_accuracy: 0.8031
Epoch 5/10
215/215 [==============================] - 2s 12ms/step - loss: 0.3830 - accuracy: 0.8295 - val_loss: 0.4461 - val_accuracy: 0.8071
Epoch 6/10
215/215 [==============================] - 3s 12ms/step - loss: 0.3763 - accuracy: 0.8327 - val_loss: 0.4483 - val_accuracy: 0.8031
Epoch 7/10
215/215 [==============================] - 3s 

In [307]:
# Make predictions
model_6_pred_probs = model_6.predict(validation_sentences)
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_results = calculate_results(y_true=val_labels,
                                    y_pred = model_6_preds)

24/24 [==============================] - 1s 9ms/step


In [308]:
model_6_results

{'accuracy': 81.23359580052494,
 'precision': 0.8127204931499284,
 'recall': 0.8123359580052494,
 'f1': 0.8105368454209699}

## Model 7: TF Hub Pretrained USE but with 10% of training data


In [309]:
# Create subsets of 10% of training data
train_10_percent = train_df_shuffled[["text", "target"]].sample(frac=0.1, random_state=42)

In [310]:
train_10_percent

,text,target
2301,Nah but srsly b4 u demolish ur partner's face ...,0
6615,OMEGA MEN Writer Explores Terrorism Religion I...,0
6619,Really cannot condemn an entire group based on...,1
974,?? New Ladies Shoulder Tote #Handbag Faux Leat...,1
5357,it's don't panic,0
...,...,...
3454,holy crap @KingMyth1999 my phone just exploded...,0
2846,Angry Woman Openly Accuses NEMA Of Stealing Re...,0
4874,Nazi Mass Murderer Became Chairman At Vaccine ...,1
425,Video Captures Man Removing American Flag From...,0


In [311]:
len(train_10_percent)/len(train_df_shuffled)

0.09996059372126626

In [312]:
train_sentences_10_percent = train_10_percent["text"].to_list()
train_labels_10_percent = train_10_percent["target"].to_list()


In [313]:
train_10_percent["target"].value_counts()

target
0    452
1    309
Name: count, dtype: int64

In [314]:
train_df_shuffled["target"].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

 To recreate a model the same as a previous model you've created you can use the `tf.keras.model.clone_model()`

In [316]:
model_7 = tf.keras.models.clone_model(model_6)
model_7.compile(loss="binary_crossentropy",
                 optimizer=tf.keras.optimizers.Adam(),
                 metrics=["accuracy"])
model_7.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_16 (Dense)            (None, 64)                32832     
                                                                 
 dense_17 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [319]:
model_7.fit(train_sentences_10_percent,
            train_labels_10_percent,
            epochs=5,
            validation_data=(validation_sentences, val_labels),
            callbacks=[create_tensorboard_callback(SAVE_DIR, "tf_hub_sentence_encoder_10_percent")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder_10_percent/20240509-142621
Epoch 1/5
24/24 [==============================] - 6s 81ms/step - loss: 0.6675 - accuracy: 0.6846 - val_loss: 0.6380 - val_accuracy: 0.7598
Epoch 2/5
24/24 [==============================] - 1s 41ms/step - loss: 0.5956 - accuracy: 0.8081 - val_loss: 0.5685 - val_accuracy: 0.7992
Epoch 3/5
24/24 [==============================] - 1s 25ms/step - loss: 0.5231 - accuracy: 0.8147 - val_loss: 0.5156 - val_accuracy: 0.8071
Epoch 4/5
24/24 [==============================] - 0s 21ms/step - loss: 0.4709 - accuracy: 0.8160 - val_loss: 0.4858 - val_accuracy: 0.8123
Epoch 5/5
24/24 [==============================] - 0s 20ms/step - loss: 0.4340 - accuracy: 0.8331 - val_loss: 0.4697 - val_accuracy: 0.8084


In [321]:
model_7_pred_probs = model_7.predict(validation_sentences)
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_results = calculate_results(y_true=val_labels,
                                    y_pred = model_7_preds)
model_7_results

24/24 [==============================] - 1s 20ms/step


{'accuracy': 80.83989501312337,
 'precision': 0.8084333684258599,
 'recall': 0.8083989501312336,
 'f1': 0.8067484900456525}